In [38]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os

from langchain.embeddings.openai import OpenAIEmbeddings

from sentence_transformers import SentenceTransformer

/Users/athrunsun/Documents/Project/LLM/RecChatbot/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
# Load env vars
load_dotenv('.env')

True

In [47]:
df = pd.read_csv('articles.csv')
df['combined_desc'] = 'product name: ' + df['prod_name'] + '; product type: ' + df['product_type_name'] \
                        + '; product group: ' + df['product_group_name'] + '; graphical appearance: ' + df['graphical_appearance_name']\
                        + '; color: ' + df['colour_group_name'] + '; garment group: ' + df['garment_group_name'] + '; detailed description: '\
                        + df['detail_desc']
df.head(10)

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,garment_group_name,detail_desc,combined_desc
0,695255001,Siv t-shirt,T-shirt,Garment Upper body,All over pattern,Dark Blue,Jersey Fancy,Short-sleeved top in soft viscose jersey with ...,product name: Siv t-shirt; product type: T-shi...
1,821115007,RICHIE SKIRT,Skirt,Garment Lower body,Check,Pink,Skirts,"Short, pleated skirt in woven fabric with a hi...",product name: RICHIE SKIRT; product type: Skir...
2,553238008,THORN LS TEE,Sweater,Garment Upper body,Solid,White,Jersey Basic,"Wide, long-sleeved top in soft cotton jersey w...",product name: THORN LS TEE; product type: Swea...
3,627147001,Bling Me Up Push,Bikini top,Swimwear,Lace,Dark Red,Swimwear,"Fully lined bikini top with hole-patterned, un...",product name: Bling Me Up Push; product type: ...
4,794520001,Plus seam at back 1p tights,Underwear Tights,Socks & Tights,Solid,Black,Socks and Tights,Tights with a seam down the back of the legs. ...,product name: Plus seam at back 1p tights; pro...
5,697564030,KELLY SHIRT S.0,Shirt,Garment Upper body,Stripe,Blue,Blouses,"Shirt in airy cotton with a collar, buttons do...",product name: KELLY SHIRT S.0; product type: S...
6,698715001,Emma Fancy Slipon SG,Sneakers,Shoes,Placement print,Light Pink,Shoes,Slip-on trainers in cotton canvas with a print...,product name: Emma Fancy Slipon SG; product ty...
7,526562001,Sonny jogger,Trousers,Garment Lower body,Solid,Black,Trousers,Pull-on trousers in stretch cotton twill with ...,product name: Sonny jogger; product type: Trou...
8,649099001,Polkagris,Shorts,Garment Lower body,Stripe,Red,Shorts,Shorts in a viscose and cotton weave containin...,product name: Polkagris; product type: Shorts;...
9,762061003,2P S/J pants,Trousers,Garment Lower body,All over pattern,Light Turquoise,Jersey Basic,Trousers in soft organic cotton jersey with an...,product name: 2P S/J pants; product type: Trou...


In [48]:
print(df['combined_desc'][0])

product name: Siv t-shirt; product type: T-shirt; product group: Garment Upper body; graphical appearance: All over pattern; color: Dark Blue; garment group: Jersey Fancy; detailed description: Short-sleeved top in soft viscose jersey with a unique nursing feature. The design includes a double layer at the top to help retain warmth while allowing easier nursing access.


In [33]:
embeddings = OpenAIEmbeddings(
    deployment="azure-text-embedding-ada-002",
    openai_api_base=os.environ['OPENAI_API_BASE'],
    openai_api_type=os.environ['OPENAI_API_TYPE'],
    openai_api_version=os.environ['OPENAI_API_VERSION'],
    openai_api_key=os.environ['OPENAI_API_KEY']
)
text = "This is a test query."
query_result = embeddings.embed_query(text)
print(len(query_result))

1536


In [49]:
"""
# tried to get embeddings through the provided API but got RateLimitError.
import time

def get_embeddings_with_retry(text):
    while True:
        try:
            return embeddings.embed_query(text)
        except Exception as e:
            if 'RateLimitError' in str(e).lower():
                time.sleep(7)  # Wait for 7 seconds before retrying
            else:
                raise e  # If it's not a rate limit error, raise the error

df['embeddings'] = df['detail_desc'].apply(lambda x: get_embeddings_with_retry(x))
"""


"\n# tried to get embeddings through the provided API but got RateLimitError.\nimport time\n\ndef get_embeddings_with_retry(text):\n    while True:\n        try:\n            return embeddings.embed_query(text)\n        except Exception as e:\n            if 'RateLimitError' in str(e).lower():\n                time.sleep(7)  # Wait for 7 seconds before retrying\n            else:\n                raise e  # If it's not a rate limit error, raise the error\n\ndf['embeddings'] = df['detail_desc'].apply(lambda x: get_embeddings_with_retry(x))\n"

In [42]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_test = model.encode(["Hello World"])
print(len(embeddings_test[0]))

384


In [43]:
# for testing
#df = df.head(10)

In [50]:
list_descriptin = df['combined_desc'].tolist()
list_embeddings = model.encode(list_descriptin)

In [51]:
df['embeddings'] = list_embeddings.tolist()
df.head(10)

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,garment_group_name,detail_desc,combined_desc,embeddings
0,695255001,Siv t-shirt,T-shirt,Garment Upper body,All over pattern,Dark Blue,Jersey Fancy,Short-sleeved top in soft viscose jersey with ...,product name: Siv t-shirt; product type: T-shi...,"[-0.0713086873292923, 0.06144961342215538, 0.0..."
1,821115007,RICHIE SKIRT,Skirt,Garment Lower body,Check,Pink,Skirts,"Short, pleated skirt in woven fabric with a hi...",product name: RICHIE SKIRT; product type: Skir...,"[-0.004989102017134428, 0.06276149302721024, 0..."
2,553238008,THORN LS TEE,Sweater,Garment Upper body,Solid,White,Jersey Basic,"Wide, long-sleeved top in soft cotton jersey w...",product name: THORN LS TEE; product type: Swea...,"[-0.054912738502025604, 0.039564747363328934, ..."
3,627147001,Bling Me Up Push,Bikini top,Swimwear,Lace,Dark Red,Swimwear,"Fully lined bikini top with hole-patterned, un...",product name: Bling Me Up Push; product type: ...,"[-0.07479564845561981, -0.017786933109164238, ..."
4,794520001,Plus seam at back 1p tights,Underwear Tights,Socks & Tights,Solid,Black,Socks and Tights,Tights with a seam down the back of the legs. ...,product name: Plus seam at back 1p tights; pro...,"[-0.1038898304104805, 0.014908007346093655, 0...."
5,697564030,KELLY SHIRT S.0,Shirt,Garment Upper body,Stripe,Blue,Blouses,"Shirt in airy cotton with a collar, buttons do...",product name: KELLY SHIRT S.0; product type: S...,"[-0.03835904970765114, 0.020938193425536156, 0..."
6,698715001,Emma Fancy Slipon SG,Sneakers,Shoes,Placement print,Light Pink,Shoes,Slip-on trainers in cotton canvas with a print...,product name: Emma Fancy Slipon SG; product ty...,"[-0.19707977771759033, -0.05739953741431236, 0..."
7,526562001,Sonny jogger,Trousers,Garment Lower body,Solid,Black,Trousers,Pull-on trousers in stretch cotton twill with ...,product name: Sonny jogger; product type: Trou...,"[-0.1269615888595581, 0.07615883648395538, 0.0..."
8,649099001,Polkagris,Shorts,Garment Lower body,Stripe,Red,Shorts,Shorts in a viscose and cotton weave containin...,product name: Polkagris; product type: Shorts;...,"[-0.005323807708919048, 0.137112557888031, -0...."
9,762061003,2P S/J pants,Trousers,Garment Lower body,All over pattern,Light Turquoise,Jersey Basic,Trousers in soft organic cotton jersey with an...,product name: 2P S/J pants; product type: Trou...,"[-0.053962480276823044, 0.022101987153291702, ..."


In [52]:
df.shape

(1000, 10)

In [53]:
df.to_csv('articles_embedding.csv')